In [1]:
from nltk.corpus import *
from nltk.stem import *
from konlpy.tag import Kkma
from nltk.tokenize import *
from nltk.tag import *
import nltk
from keras.utils import to_categorical
import numpy as np
from keras import *
from keras.datasets import mnist
from collections import Counter
import tensorflow as tf

text="Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.[27] Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is often described as a 'batteries included' language due to its comprehensive standard library.[28]"
text=sent_tokenize(text)
text

Using TensorFlow backend.


['Python is an interpreted, high-level, general-purpose programming language.',
 "Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
 'Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.',
 '[27] Python is dynamically typed and garbage-collected.',
 'It supports multiple programming paradigms, including procedural, object-oriented, and functional programming.',
 "Python is often described as a 'batteries included' language due to its comprehensive standard library.",
 '[28]']

In [2]:
voc=Counter()
sentences = []
stop_words = stopwords.words('english')
for i in text:
    sentence=word_tokenize(i)
    res = []
    for word in sentence: 
        word=word.lower()
        if word not in stop_words: 
            if len(word) > 2:
                res.append(word)
                voc[word]=voc[word]+1
    sentences.append(res) 
print(sentences)

[['python', 'interpreted', 'high-level', 'general-purpose', 'programming', 'language'], ['created', 'guido', 'van', 'rossum', 'first', 'released', '1991', 'python', 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'notable', 'use', 'significant', 'whitespace'], ['language', 'constructs', 'object-oriented', 'approach', 'aim', 'help', 'programmers', 'write', 'clear', 'logical', 'code', 'small', 'large-scale', 'projects'], ['python', 'dynamically', 'typed', 'garbage-collected'], ['supports', 'multiple', 'programming', 'paradigms', 'including', 'procedural', 'object-oriented', 'functional', 'programming'], ['python', 'often', 'described', "'batteries", 'included', 'language', 'due', 'comprehensive', 'standard', 'library'], []]


In [3]:
voc

Counter({'python': 4,
         'interpreted': 1,
         'high-level': 1,
         'general-purpose': 1,
         'programming': 3,
         'language': 3,
         'created': 1,
         'guido': 1,
         'van': 1,
         'rossum': 1,
         'first': 1,
         'released': 1,
         '1991': 1,
         'design': 1,
         'philosophy': 1,
         'emphasizes': 1,
         'code': 2,
         'readability': 1,
         'notable': 1,
         'use': 1,
         'significant': 1,
         'whitespace': 1,
         'constructs': 1,
         'object-oriented': 2,
         'approach': 1,
         'aim': 1,
         'help': 1,
         'programmers': 1,
         'write': 1,
         'clear': 1,
         'logical': 1,
         'small': 1,
         'large-scale': 1,
         'projects': 1,
         'dynamically': 1,
         'typed': 1,
         'garbage-collected': 1,
         'supports': 1,
         'multiple': 1,
         'paradigms': 1,
         'including': 1,
         'proc

In [5]:
voc.items()

dict_items([('python', 4), ('interpreted', 1), ('high-level', 1), ('general-purpose', 1), ('programming', 3), ('language', 3), ('created', 1), ('guido', 1), ('van', 1), ('rossum', 1), ('first', 1), ('released', 1), ('1991', 1), ('design', 1), ('philosophy', 1), ('emphasizes', 1), ('code', 2), ('readability', 1), ('notable', 1), ('use', 1), ('significant', 1), ('whitespace', 1), ('constructs', 1), ('object-oriented', 2), ('approach', 1), ('aim', 1), ('help', 1), ('programmers', 1), ('write', 1), ('clear', 1), ('logical', 1), ('small', 1), ('large-scale', 1), ('projects', 1), ('dynamically', 1), ('typed', 1), ('garbage-collected', 1), ('supports', 1), ('multiple', 1), ('paradigms', 1), ('including', 1), ('procedural', 1), ('functional', 1), ('often', 1), ('described', 1), ("'batteries", 1), ('included', 1), ('due', 1), ('comprehensive', 1), ('standard', 1), ('library', 1)])

In [14]:
voc_sorted=sorted(voc.items(),key=lambda a:a[1], reverse=True)
voc_sorted

# res={} # 빈도수가 2이상인 데이터 저장
# i=0
# for (w,f) in voc_sorted:
#     if f > 1:
#         i+=1
#         res[w]=i
# print(res) # 번호는 인덱스를 의미 

[('python', 4),
 ('programming', 3),
 ('language', 3),
 ('code', 2),
 ('object-oriented', 2),
 ('interpreted', 1),
 ('high-level', 1),
 ('general-purpose', 1),
 ('created', 1),
 ('guido', 1),
 ('van', 1),
 ('rossum', 1),
 ('first', 1),
 ('released', 1),
 ('1991', 1),
 ('design', 1),
 ('philosophy', 1),
 ('emphasizes', 1),
 ('readability', 1),
 ('notable', 1),
 ('use', 1),
 ('significant', 1),
 ('whitespace', 1),
 ('constructs', 1),
 ('approach', 1),
 ('aim', 1),
 ('help', 1),
 ('programmers', 1),
 ('write', 1),
 ('clear', 1),
 ('logical', 1),
 ('small', 1),
 ('large-scale', 1),
 ('projects', 1),
 ('dynamically', 1),
 ('typed', 1),
 ('garbage-collected', 1),
 ('supports', 1),
 ('multiple', 1),
 ('paradigms', 1),
 ('including', 1),
 ('procedural', 1),
 ('functional', 1),
 ('often', 1),
 ('described', 1),
 ("'batteries", 1),
 ('included', 1),
 ('due', 1),
 ('comprehensive', 1),
 ('standard', 1),
 ('library', 1)]

### keras를 활용한 문자열 처리

In [15]:
# 케라스 문자열 처리
from keras.preprocessing.text import Tokenizer
tok=Tokenizer() # 객체 생성
tok.fit_on_texts(text)

In [16]:
# 단어 단위로 인덱스와 함께 출력
print(tok.word_index) # 정렬(빈도수)-> 인덱싱

{'and': 1, 'python': 2, 'is': 3, 'programming': 4, 'language': 5, 'its': 6, 'code': 7, 'object': 8, 'oriented': 9, 'to': 10, 'an': 11, 'interpreted': 12, 'high': 13, 'level': 14, 'general': 15, 'purpose': 16, 'created': 17, 'by': 18, 'guido': 19, 'van': 20, 'rossum': 21, 'first': 22, 'released': 23, 'in': 24, '1991': 25, "python's": 26, 'design': 27, 'philosophy': 28, 'emphasizes': 29, 'readability': 30, 'with': 31, 'notable': 32, 'use': 33, 'of': 34, 'significant': 35, 'whitespace': 36, 'constructs': 37, 'approach': 38, 'aim': 39, 'help': 40, 'programmers': 41, 'write': 42, 'clear': 43, 'logical': 44, 'for': 45, 'small': 46, 'large': 47, 'scale': 48, 'projects': 49, '27': 50, 'dynamically': 51, 'typed': 52, 'garbage': 53, 'collected': 54, 'it': 55, 'supports': 56, 'multiple': 57, 'paradigms': 58, 'including': 59, 'procedural': 60, 'functional': 61, 'often': 62, 'described': 63, 'as': 64, 'a': 65, "'batteries": 66, "included'": 67, 'due': 68, 'comprehensive': 69, 'standard': 70, 'libra

In [17]:
print(tok.word_counts) # 정렬(빈도수)

OrderedDict([('python', 3), ('is', 3), ('an', 1), ('interpreted', 1), ('high', 1), ('level', 1), ('general', 1), ('purpose', 1), ('programming', 3), ('language', 3), ('created', 1), ('by', 1), ('guido', 1), ('van', 1), ('rossum', 1), ('and', 5), ('first', 1), ('released', 1), ('in', 1), ('1991', 1), ("python's", 1), ('design', 1), ('philosophy', 1), ('emphasizes', 1), ('code', 2), ('readability', 1), ('with', 1), ('its', 3), ('notable', 1), ('use', 1), ('of', 1), ('significant', 1), ('whitespace', 1), ('constructs', 1), ('object', 2), ('oriented', 2), ('approach', 1), ('aim', 1), ('to', 2), ('help', 1), ('programmers', 1), ('write', 1), ('clear', 1), ('logical', 1), ('for', 1), ('small', 1), ('large', 1), ('scale', 1), ('projects', 1), ('27', 1), ('dynamically', 1), ('typed', 1), ('garbage', 1), ('collected', 1), ('it', 1), ('supports', 1), ('multiple', 1), ('paradigms', 1), ('including', 1), ('procedural', 1), ('functional', 1), ('often', 1), ('described', 1), ('as', 1), ('a', 1), ("'

In [18]:
print(tok.texts_to_sequences(text)) # tok.word_index의 인덱스를 가지고 원래는 문장 순서대로 출력

[[2, 3, 11, 12, 13, 14, 15, 16, 4, 5], [17, 18, 19, 20, 21, 1, 22, 23, 24, 25, 26, 27, 28, 29, 7, 30, 31, 6, 32, 33, 34, 35, 36], [6, 5, 37, 1, 8, 9, 38, 39, 10, 40, 41, 42, 43, 44, 7, 45, 46, 1, 47, 48, 49], [50, 2, 3, 51, 52, 1, 53, 54], [55, 56, 57, 4, 58, 59, 60, 8, 9, 1, 61, 4], [2, 3, 62, 63, 64, 65, 66, 67, 5, 68, 10, 6, 69, 70, 71], [72]]


In [23]:
tok.word_counts.items()
# words_freq=빈도수가 2보다 작은 단어들을 저장
words_freq=[w for w,c in tok.word_counts.items() if c<2]
words_freq

['an',
 'interpreted',
 'high',
 'level',
 'general',
 'purpose',
 'created',
 'by',
 'guido',
 'van',
 'rossum',
 'first',
 'released',
 'in',
 '1991',
 "python's",
 'design',
 'philosophy',
 'emphasizes',
 'readability',
 'with',
 'notable',
 'use',
 'of',
 'significant',
 'whitespace',
 'constructs',
 'approach',
 'aim',
 'help',
 'programmers',
 'write',
 'clear',
 'logical',
 'for',
 'small',
 'large',
 'scale',
 'projects',
 '27',
 'dynamically',
 'typed',
 'garbage',
 'collected',
 'it',
 'supports',
 'multiple',
 'paradigms',
 'including',
 'procedural',
 'functional',
 'often',
 'described',
 'as',
 'a',
 "'batteries",
 "included'",
 'due',
 'comprehensive',
 'standard',
 'library',
 '28']

In [25]:
for w in words_freq:
    print(tok.word_index[w]) # 각 단어 인덱스
    print(tok.word_counts[w]) # 각 단어 빈도수

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [26]:
for w in words_freq:
    del tok.word_index[w] # 각 단어 인덱스
    del tok.word_counts[w] # 각 단어 빈도수
print(tok.texts_to_sequences(text))
print("="*30)
print(tok.word_index)

[[2, 3, 4, 5], [1, 7, 6], [6, 5, 1, 8, 9, 10, 7, 1], [2, 3, 1], [4, 8, 9, 1, 4], [2, 3, 5, 10, 6], []]
{'and': 1, 'python': 2, 'is': 3, 'programming': 4, 'language': 5, 'its': 6, 'code': 7, 'object': 8, 'oriented': 9, 'to': 10}


In [33]:
words=['java','python','ds','ds','java','java']
from nltk import FreqDist
fd=FreqDist(words)
fd.N()
fd.freq('java') # 0.5 확률
fd['ds'] # 단어 2번 등장
fd.most_common(1) # 가장 빈도수가 높은 단어 추출
fd.most_common(2) # 가장 빈도수가 높은 2개 단어 추출

[('java', 3), ('ds', 2)]

In [38]:
# one-hot 인코딩
from konlpy.tag import Okt
okt=Okt()
tok=okt.morphs("나는 자연어 자연어 처리를 공부하고 있습니다")
tok

['나', '는', '자연어', '자연어', '처리', '를', '공부', '하고', '있습니다']

In [39]:
word2Idx={}
for t in tok:
    if t not in word2Idx.keys():
        word2Idx[t] = len(word2Idx)
print(word2Idx)
# 중복된 단어는 추가 X

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '공부': 5, '하고': 6, '있습니다': 7}


### 문자열 원핫인코딩

In [44]:
def ohe(w,word2Idx):
    ohv=[0]*len(word2Idx)
    p=word2Idx[w]
    ohv[p]=1
    return ohv
print(ohe("공부", word2Idx)) # 000000100 출력

[0, 0, 0, 0, 0, 1, 0, 0]


In [48]:
# 케라스 원핫인코딩
text="오늘 메뉴는 뼈다귀 해장국입니다. 맛있게 먹어요. 국산이래요. 뼈다귀 최고 뼈다귀 최고"

In [50]:
from keras_preprocessing.text import Tokenizer
tok=Tokenizer()
tok.fit_on_texts([text])
tok.word_counts
tok.word_index

{'뼈다귀': 1,
 '최고': 2,
 '오늘': 3,
 '메뉴는': 4,
 '해장국입니다': 5,
 '맛있게': 6,
 '먹어요': 7,
 '국산이래요': 8}

In [55]:
text2="뼈다귀 관련 음식을 가장 맛있게 먹어요"
res=tok.texts_to_sequences([text2]) # text2를 기존 코퍼스에 대입해서 동일한 부분의 인덱스 출력
res
#res[0][1] => 6 : 요소 하나씩 참고하고 싶을 때 쓰면 좋음

[[1, 6, 7]]

In [61]:
# 1 -> 01000000
# 6 -> 00000010
vLen=len(tok.word_index)
from keras.utils import to_categorical
pres=to_categorical(res, num_classes=vLen+1) 
# to_categorical함수는 0부터 시작하는데 데이터에는 0이 없어 하나 추가하기
pres

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.]]], dtype=float32)

#### 원핫인코딩 문제점
1) 음식, 식사, 점심, 날씨 => 원핫인코딩 => 크기:4
    => 단어들간에 유사한 정도를 나타내기 어려움
    
2) 공간을 많이 차지(sparse matrix => dense matrix 변환 필요) 

3) 연관검색어 표현이 어려움. ex) '뼈다귀' => '뼈다귀 해장국', '뼈다귀 국물', '뼈다귀 감자탕'...  
    => 해결하기 위한 여러가지 노력(ing)=>LSA, Word2Vec...  
    => LSA:카운트 기반으로 단어의 의미를 벡터화 하는 알고리즘  
    => Word2Vec:단어를 벡터 공간으로 표현(거리)  
    => Seq2Seq, RNNLM...

In [ ]:
# 언어 모델?
# Vocabulary: 기계가 학습한 단어의 집합
# out of Vocabulary(OOV) : 기계가 학습 안한 단어들
# => BPE, WPM 알고리즘(데이터 전처리)

#### BPE : 연속으로 가장 많이 등장한 글자의 쌍 => 한 글자로 표현(변환)  
   - 문자열 : aaabadaaabac   
      Z=aa  
   - 문자열 : ZabadZabac  
      Y=ab
   - 문자열 : ZYadZYac
      X=ZY
   - 문자열 : XadXac
      W=Xa
   - 문자열 : WdWc
------------------------------------------
- 허프만 인코딩(트리)가 효율 짱 좋음  
    {'low':5, 'lower':2, 'newest':6, 'widest':3}  
    (단어 : 빈도수,...}  
-------------------------------------------
- BPE 알고리즘 사용 -> OOV에 대한 문제점 해결  
     food : 의미를 이해할 수 없음
     lowest : 의미를 이해할 수 있음 (by BPE)  
※ KMP, 보이어무어, 레벤슈타인 거리 등 검색  
1) 모든 단어들을 글자로 분리   
    {'l o w': 5, 'l o w e r' : 2, 'n e w e s t':6, 'w i d e s t' : 3} => 빈도수   
    -> l,o,w,e,r,n,w,s,t,i,d   
2) 빈도가 가장 높은 문자들의 쌍을 검색  
    -> es가 가장 많이 등장(9번) => es를 묶음  
    {'l o w': 5, 'l o w e r' : 2, 'n e w es t':6, 'w i d es t' : 3}
    -> l,o,w,e,r,n,w,s,t,i,d,es
3) 빈도가 가장 높은 문자들의 쌍을 검색  
    -> es t가 가장 많이 등장(9번) => est를 묶음  
    {'l o w': 5, 'l o w e r' : 2, 'n e w est':6, 'w i d est' : 3}  
    -> l,o,w,e,r,n,w,s,t,i,d,es,est  
4) 빈도가 가장 높은 문자들의 쌍을 검색    
    -> l o 가 가장 많이 등장(7번) => lo를 묶음    
    {'l o w': 5, 'l o w e r' : 2, 'n e w est':6, 'w i d est' : 3}  
    -> l,o,w,e,r,n,w,s,t,i,d,es,est, lo  

=> 위 과정을 반복...(10번)   
    -> l,o,w,e,r,n,s,t,i,d,  
    -> 9번 등장 : es,est,   
    -> 7번 등장 : lo, low,   
    -> 6번 등장 : ne, new, newest,   
    -> 3번 등장 : wi, wid, widest...   
    
- 'lowest'란 단어가 등장한다면,기계는 우선 'lowest'를 전부 글자 단위로 분할. 즉, 'l, o, w, e, s, t'가 됨.   
그리고 기계는 위의 단어 집합을 참고로 하여 'low'와 'est'를 찾아냄.  
단어를 찾는건 앞 또는 뒤에서부터 찾아도 됨.  

In [95]:
# 1. BFE 알고리즘 구현(연습문제)
# 2. 상품분류(10가지) 미니 프로젝트 
# gspark@kw.ac.kr